# Alternative Evaluation Metrics and Custom Feature Selection
Some classical evaluation metrics occasionally fall short under certain circumstances. In the first part of this homework, you will implement a few special evaluation metrics used in predictive analytics that is designated for one-sided and imbalanced prediction tasks.

In the second part, you will be implementing a custom feature selection procedure. This will be a univariate feature selection method. You will be given a toy dataset called 'Car Evaluation Data Set' (see: http://archive.ics.uci.edu/ml/datasets/Car+Evaluation for details). You are not required to, but advised to test your code with the toy dataset, or any other dataset that contains categorical variables.


## Part 1: Forecast Metrics
There are many types of forecasts, each calling for slightly different methods of verification. The primary target for the measures below are dichotomous forecasts (in other words, binary predictions). 

The forecast terminology is slightly different. Following are the differences:
- Instead of True Positives (TP) --> _Hits_ (a) is used, 
- Instead of False Positives (FP) --> _False Alarms_ (b) is used,
- Instead of False Negatives (FN) --> _Misses_ (c) is used, 
- Instead of True Negatives (TN) --> _Correct Rejects_ (d) is used.

For every single model run, you are given:
1. a set of observations (Event [1] or No Event [0]) 
2. a set of prediction scores (a float between 0 and 1) and an event threshold, where the predicted outcome will be 

<center>$\hat{y}= 
\begin{cases}
    1 \text{ (Event occurred)},& \text{if } score \geq threshold\\
    0 \text{ (No event),}      & \text{if } score < threshold
\end{cases} $ <center>

Note that observations are ground truth and prediction scores and threshold will be used for determining the predicted model output.

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Here are some test data you can use for this homework
Y_test = [0, 0, 1, 1, 0, 1, 0, 0, 0, 1] # observations
P_scores = [0.1, 0.32, 0.48, 0.9, 0.6, 0.55, 0.42, 0.37, 0.61, 0.66] # prediction scores
threshold = 0.5 # prediction threshold

### Part 1 - Question 1 (10 points)
Given prediction scores, threshold, and observations create a function to determine the elements of a confusion matrix. For ease of use, you will output a `dict` (dictionary) object instead of a 2-dimensional numpy array. Note that _positive_ corresponds to the event occurrence.

In [3]:
def binary_conf_matrix(observation, p_scores, threshold):
    predictions = []

    #filling in a list of prediction scores based on the threshold 
    for p in p_scores:
        if p >= threshold:
            predictions.append(1)
        else:
            predictions.append(0)
    
    TP = 0
    TN = 0
    FP = 0
    FN = 0

    #iterate through list of predictions and find elements of a confusion matrix
    for i in range(len(predictions)):
        if(observation[i] ==1 ):
            if(predictions[i] == 1):
                TP +=1
            else:
                FP +=1
        else:
            if(predictions[i] == 0):
                TN +=1
            else:
                FN +=1
    
    return {'TP':TP, 'FP':FP, 'TN':TN, 'FN':FN}


bc_matrix = binary_conf_matrix(Y_test, P_scores, threshold)
bc_matrix

{'TP': 3, 'FP': 1, 'TN': 4, 'FN': 2}

### Part 1 - Question 2 (10 points)

Create functions for calculating accuracy, precision, recall, and F1-score. You can use the definitions from slides (Chapter 11). (You are supposed to calculate the precision and recall (and thus F1-score) for 'Event' [1] class.)

In [4]:
#accuracy = (tp+tn)/(tp+tn+tp+tn)
def accuracy(observation, p_scores, threshold):
    bc_matrix = binary_conf_matrix(observation, p_scores, threshold)
    TP, TN, FP, FN = bc_matrix['TP'], bc_matrix['TN'],bc_matrix['FP'], bc_matrix['FN']
    return (TP+TN)/(TP+TN+FP+FN)

#precision = tp/(tp+fp)    
def precision(observation, p_scores, threshold):
    bc_matrix = binary_conf_matrix(observation, p_scores, threshold)
    TP,FP= bc_matrix['TP'],bc_matrix['FP']
    return TP/(TP+FP)

#recall = tp/(tp+fn)
def recall(observation, p_scores, threshold):
    bc_matrix = binary_conf_matrix(observation, p_scores, threshold)
    TP,FN = bc_matrix['TP'],bc_matrix['FN']
    return TP/(TP+FN)

#f1= 2*((precision*recall)/(precision + recall))
def f1score(observation, p_scores, threshold):
    precis = precision(observation, p_scores, threshold )
    rec = recall( observation, p_scores, threshold )
    return 2*(precis*rec)/(precis+rec)

    
print("accuracy =", accuracy( Y_test, P_scores, threshold ))

print("precision =",precision( Y_test, P_scores, threshold ))

print("recall =",recall( Y_test, P_scores, threshold ))

print("f1score =", f1score( Y_test, P_scores, threshold ))

accuracy = 0.7
precision = 0.75
recall = 0.6
f1score = 0.6666666666666665


### Part 1 - Question 3 (5 points)
Calculate the bias score (BIAS). Bias score measures the ratio of the frequency of predicted event occurrences to the frequency of observed events. It can be calculated using the following formula:

$ BIAS = \frac{\text{hits} + \text{false alarms} }{ \text{hits} + \text{misses} }$

In [5]:
def bias_score(observation, p_scores, threshold):
    bc_matrix = binary_conf_matrix(observation, p_scores, threshold)
    TP, TN, FP, FN = bc_matrix['TP'], bc_matrix['TN'],bc_matrix['FP'], bc_matrix['FN']
    return (TP+FP)/(TP+FN)

    
print("BIAS =", bias_score(Y_test, P_scores, threshold))

BIAS = 0.8


### Part 1 - Question 4 (5 points)

Calculate the false alarm ratio (FAR). FAR is an indicator of what fraction of the predicted events actually did not occur (i.e., they were false alarms)? You can calculate the FAR as follows: 

$ FAR = \frac{ \text{false alarms} }{ \text{hits} + \text{false alarms} }  $

In [6]:
def far(observation, p_scores, threshold):
    TP,FP = bc_matrix['TP'],bc_matrix['FP']
    return FP/(TP+FP)

print("FAR =", far(Y_test, P_scores, threshold))

FAR = 0.25


### Part 1 - Question 5 (5 points)
Calculate the threat score. Threat score (which is also referred to as critical success index -- CSI) indicates how well did the predicted event outcomes correspond to the observed events. It measures the fraction of actual __and/or__ predicted events that were correctly predicted. It can be thought of as the accuracy when the correct negatives (TN) have been removed from consideration. It can be calculated as follows:

$CSI = \frac{ \text{hits} }{ \text{hits} + \text{misses} + \text{false alarms} } $



In [7]:
def csi(observation, p_scores, threshold):
    TP, FP, FN = bc_matrix['TP'], bc_matrix['FP'], bc_matrix['FN']
    return TP/(TP+FN+FP)


print("CSI =",csi (Y_test, P_scores, threshold))

CSI = 0.5


### Part 1 - Question 6 (7.5 points)
Skill scores are often used to understand the improvement of model performance over a given baseline (often a hypothetical, predetermined random forecast result). The first skill score you will implement is _true skill statistic (TSS)_ (which is also known as Hanssen-Kuipers Skill Score. It can be used to understand how well the prediction model separate the events (detected) from the no events. TSS can be calculated as follows:

$ TSS = \frac{\text{hits} }{\text{hits} + \text{misses}} - \frac{\text{false alarms} }{\text{false alarms} + \text{correct negatives}} $

In [8]:
def tss(observation, p_scores, threshold):
     TP, TN, FP, FN = bc_matrix['TP'], bc_matrix['TN'],bc_matrix['FP'], bc_matrix['FN']
     return (TP/(TP+FN)) - (FP/(FP+TN))


print("tss =", tss(Y_test, P_scores, threshold))

tss = 0.39999999999999997


### Part 1 - Question 7 (7.5 points)
The next skill score to calculate is Gilbert Skill Score (also known as Equitable Threat Score). This is an interesting indicator of performance because it measures the fraction of observed and/or predicted events that were correctly predicted, adjusted for hits (correctly predicted events) associated with random chance. For example, it is easier to correctly predict rain occurrence in a wet climate than in a dry climate. In other words, GSS can answer how well the predicted event occurrences correspond to the observed events (accounting for correct predictions appearing due to chance). The Gilbert Skill Score can be calculated as follows:

$GSS = \frac{ \text{hits} - \text{hits}_{random}}{ \text{hits} + \text{misses} + \text{false alarms} - \text{hits}_{random} } $

where the random hits can ba calculated as:

$ \text{hits}_{random} = \frac{ (\text{hits}+\text{misses} )* (\text{hits}+\text{false alarms} )}{total} $.

_total_ is sum of all the elements in confusion matrix. 

Hint: Notice the similarity between GSS and threat score.

In [9]:
def gss(observation, p_scores, threshold):
    TP, TN, FP, FN = bc_matrix['TP'], bc_matrix['TN'],bc_matrix['FP'], bc_matrix['FN']
    hits_random = (TP + FN)*(TP+FP) / (TP+TN+FP+FN)
    return (TP-hits_random)/(TP+FN+FP-hits_random)

print("gss =", gss(Y_test, P_scores, threshold))

gss = 0.25


### Part 1 - Bonus Question (10 points)

Your last task is to determine an optimal threshold based on prediction scores, observations, and a given performance measure. Create a function called `pick_threshold` which will pick the best prediction score threshold (that will return the highest performance measure based on the given performance metric). Hint: Python allows you to pass a (performance measure) function (such as `tss`, `csi`, or `f1score`) to `pick_threshold`. 

In [79]:
def pick_threshold(observation, p_scores, mfunc):

    optimal_threshold = 0 
    best_score = 0

    #iterate through range of values [.01-1]
    #set score = mfunc() performance measure
    #if score is higher than best score ->swap and threshold gets set to that i 
    for i in np.arange(.01,1,0.01): 
        try :
            score =mfunc(observation, p_scores, i)
        except:
            score = 0

        if score > best_score:
            best_score = score 
            optimal_threshold = i 
            
    return optimal_threshold,best_score


gss_threshold, gss_score = pick_threshold(Y_test, P_scores, gss)
print("gss_threshold:",gss_threshold, "gss_score:",gss_score)

tss_threshold, tss_score = pick_threshold(Y_test, P_scores, tss)
print("tss_threshold:", tss_threshold, "tss_score:",tss_score)

csi_threshold, csi_score = pick_threshold(Y_test, P_scores, csi)
print("csi_threshold:", csi_threshold, "csi_score:",csi_score)

f1_threshold, f1_score = pick_threshold(Y_test, P_scores, f1score)
print("f1_threshold:", f1_threshold, "f1_score:",f1_score)


gss_threshold: 0.01 gss_score: 0.25
tss_threshold: 0.01 tss_score: 0.39999999999999997
csi_threshold: 0.01 csi_score: 0.5
f1_threshold: 0.42000000000000004 f1_score: 0.8


## Part 2: IUFS Implementation

In this part, you are advised to use the car evaluation dataset. The given dataset contains six descriptive features and a target variable. Each of those are ordinal scale, categorical variables. The name of the target feature is 'evaluation'.

Note here that you are expected to write your own code for the feature selection method. Therefore, DO NOT COPY AND PASTE CODE OR USE LIBRARY FUNCTIONS. The goal of the homework is not to see if you can call library functions but to have you practice with the impurity measures and feature selection techniques.

In [25]:
edf = pd.read_csv('careval.csv')
# edf.head()
edf.info()
edf.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   buying      1728 non-null   object
 1   maint       1728 non-null   object
 2   doors       1728 non-null   object
 3   persons     1728 non-null   object
 4   lug_boot    1728 non-null   object
 5   safety      1728 non-null   object
 6   evaluation  1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


,buying,maint,doors,persons,lug_boot,safety,evaluation
count,1728,1728,1728,1728,1728,1728,1728
unique,4,4,4,3,3,3,4
top,vhigh,vhigh,2,2,small,low,unacc
freq,432,432,432,576,576,576,1210


You will create a feature selection method called IUFS (impurity-based univariate feature selection), which will select the most informative features with a univariate filter feature selection schema. This feature selection method will take the dataset, name of the target variable, number of features to be selected (k) and the measure of impurity as an input, and will output the names of k best features based on the information gain. You are expected to implement information gain, entropy and Gini index functions. Note here that this will be a univariate selection, which means that you need to test the features individually.

### Part 2 - Question 1 (10 points)
Implement a function that calculates entropy for a feature in a given dataset.

In [35]:
# entropy (H)

def entropy(feature, dataset):
    base = 2
    en = pd.Series(dataset[feature]).value_counts(normalize=True, sort=False)
    ent = -np.sum(en * np.log(en)/np.log(base))

    return ent
    
buying_entropy = entropy('buying', edf)
print("entropy(buying) =", buying_entropy)
evaluation_entropy = entropy('evaluation', edf)
print("entropy(evaluation) = ", evaluation_entropy)

entropy(buying) = 2.0
entropy(evaluation) =  1.2057409700121753


### Part 2 - Question 3 (10 points)
Implement a function that calculates gini index for a feature in a given dataset.

In [36]:
# gini index (Gini)

def gini(feature, dataset):
    
    en = pd.Series(dataset[feature]).value_counts(normalize=True, sort=False)
    gi = 1-np.sum(en**2)
    return gi

    
evaluation_gini = gini('evaluation', edf) 
print("gini(evaluation) =", evaluation_gini)

gini(evaluation) = 0.4572837630744171


### Part 2 - Question 3 (10 points)
Implement a function that calculates information gain (IG) for a feature in a given dataset and the target variable. This function is also expected take a measure of impurity.

In [80]:
# information gain (IG)

def IG(feature, target, dataset, measure):
    #calculate impurity for entire dataset
    t_impurity = eval(measure)(target,dataset)

    e_list = list() #stores entropy og each partition
    w_list = list() #stores num.instances in each partition

    #interate through each level of the df to partition data set w/ respect to TF 
    #calculate entropy and weight the level's part. 
    for level in dataset[feature].unique():
        fdf = dataset[dataset[feature] == level]
        ent = eval(measure)(target, fdf)
        e_list.append(ent)
        w_list.append(len(fdf) / len(dataset))
        
    feature_remaining_impurity = np.sum(np.array(e_list) * np.array(w_list))
    information_gain = t_impurity - feature_remaining_impurity
    return(information_gain)

print("Information gain with Entropy: Evaluation")
eval_IG = IG('buying','evaluation', edf, 'entropy')
for i in edf.drop(columns='evaluation').columns:
    print(i,":", eval_IG )

print("\nInformation gain with Gini: Evaluation")
eval_gini = IG('buying','evaluation', edf, 'gini')
for i in edf.drop(columns='evaluation').columns:
    print(i," :", eval_gini )

Information gain with Entropy: Evaluation
buying : 0.09644896916961399
maint : 0.09644896916961399
doors : 0.09644896916961399
persons : 0.09644896916961399
lug_boot : 0.09644896916961399
safety : 0.09644896916961399

Information gain with Gini: Evaluation
buying  : 0.014286077889231918
maint  : 0.014286077889231918
doors  : 0.014286077889231918
persons  : 0.014286077889231918
lug_boot  : 0.014286077889231918
safety  : 0.014286077889231918


### Part 2 - Question 4 (20 points)
Implement the IUFS feature selection as a function that will select k most informative features using information gain based on a target variable. This function will also take a measure of impurity. It will return a list of k feature names.

In [59]:
def IUFS(target, dataset, k, measure='entropy'):
    #fill a list with all the entropy scores for each target, sort them, and pick k best features 
    f_list = []
    for f in edf.columns.drop(target):
        f_list.append((f, IG(f, target, dataset,measure)))
    arr = sorted(f_list, key = lambda x: x[1], reverse=True) [:k]
    return [x[0] for x in arr]

print("Most informative features: ",IUFS('evaluation', edf, 2, measure='entropy') )


Most informative features:  ['safety', 'persons']


### Part 2 - Bonus Question (10 points)
Improve the IUFS by including an option for gain ratio. Gain ratio is an alternative to information gain and can be used with either of the Gini index or entropy measures. First implement gain ratio (GR), then implement the updated version of IUFS function (IUFS2), which will take a selection metric ('IG' for information gain or 'GR' for gain ratio) as an argument.

In [61]:
def GR(feature, target, dataset, measure):
    #gr = Ig/measure 
    gain_ratio = IG(feature, target, dataset, measure) / eval(measure)(feature,dataset)
    return gain_ratio

print("Gain ration = ",GR('buying','evaluation', edf, 'gini')) 

Gain ration =  0.019048103852309223


In [63]:
def IUFS2(target, dataset, k, measure='entropy', gain='IG'):
    #fill a list with all the entropy scores for each target, sort them, and pick k best features 
    f_list = []
    for f in edf.columns.drop(target):
        f_list.append((f, IG(f, target, dataset,measure)))
    arr = sorted(f_list, key = lambda x: x[1], reverse=True) [:k]
    return [x[0] for x in arr]
    
print("Most informative features with Gain Ratio: ")
print(IUFS2('evaluation', edf, 2, measure='gini', gain='GR'))

Most informative features with Gain Ratio: 
['safety', 'persons']
